In [1]:
#@title Generic imports
import numpy as np
from gym import spaces
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
#@title RecSim imports
from recsim import document
from recsim import user
from recsim.choice_model import MultinomialLogitChoiceModel
from recsim.simulator import environment
from recsim.simulator import recsim_gym

In [3]:
DOC_NUM = 5

In [4]:
np.random.seed(1)

In [5]:
W = np.random.uniform(0, 1, size=(DOC_NUM, DOC_NUM))

In [6]:
def generate_W():
    W = np.random.uniform(0, 1, size=(DOC_NUM, DOC_NUM))
    W = W * (np.ones((DOC_NUM, DOC_NUM)) - np.eye(DOC_NUM, DOC_NUM))
    W = W / W.sum(axis=1).reshape(-1, 1)
    return W

In [7]:
W = generate_W()

In [8]:
W

array([[0.        , 0.07255887, 0.03332115, 0.14489761, 0.74922237],
       [0.0431104 , 0.        , 0.41989149, 0.23371334, 0.30328477],
       [0.1782122 , 0.38775524, 0.        , 0.01032974, 0.42370283],
       [0.46629066, 0.35279238, 0.13224144, 0.        , 0.04867552],
       [0.2311257 , 0.46886092, 0.15151319, 0.1485002 , 0.        ]])

### Модель документа

Модель состоит только из номера

In [9]:
class LTSDocument(document.AbstractDocument):

#    doc_num = DOC_NUM
    def __init__(self, doc_id):
        # doc_id is an integer representing the unique ID of this document
        super(LTSDocument, self).__init__(doc_id)
        
    def create_observation(self):
        return np.array([self._doc_id])

    @staticmethod
    def observation_space():
        return spaces.Discrete(LTSDocument.doc_num)
  
    def __str__(self):
        return "Document #{}".format(self._doc_id)

In [10]:
class LTSDocumentSampler(document.AbstractDocumentSampler):
    def __init__(self, doc_num=10, doc_ctor=LTSDocument, **kwargs):
        doc_ctor.doc_num = doc_num
        super(LTSDocumentSampler, self).__init__(doc_ctor, **kwargs)
        self._doc_count = 0
        self.doc_num = doc_num
        
    def sample_document(self):
        doc_features = {}
        doc_features['doc_id'] = self._doc_count
        
        if self._doc_count < self.doc_num:
            self._doc_count = (self._doc_count + 1) % self.doc_num
        else:
            self._doc_count = 0
        return self._doc_ctor(**doc_features)

With this we can now simulate documents.

In [11]:
sampler = LTSDocumentSampler(DOC_NUM)
for i in range(DOC_NUM * 2): 
    d = sampler.sample_document()
    print("Documents have observation space:", d.observation_space(), "\n"
          "An example realization is: ", d.create_observation())

Documents have observation space: Discrete(5) 
An example realization is:  [0]
Documents have observation space: Discrete(5) 
An example realization is:  [1]
Documents have observation space: Discrete(5) 
An example realization is:  [2]
Documents have observation space: Discrete(5) 
An example realization is:  [3]
Documents have observation space: Discrete(5) 
An example realization is:  [4]
Documents have observation space: Discrete(5) 
An example realization is:  [0]
Documents have observation space: Discrete(5) 
An example realization is:  [1]
Documents have observation space: Discrete(5) 
An example realization is:  [2]
Documents have observation space: Discrete(5) 
An example realization is:  [3]
Documents have observation space: Discrete(5) 
An example realization is:  [4]


## User state and user sampler 
Similarly to documents, we begin by implementing a user state class, i.e. a class that acts as a container for all these parameters. Similarly to AbstractDocument, AbstractUserState requires us to implement an observation_space() and create_observations(). These are used to feed partial (or complete) information about the user's state to the agent at every iteration. 

We also maintain a time budget, which will cap the session length. In this scenario, the session length will be fixed to some constant, so it's not worth being explicit with our time budget modeling, but one can definitely consider this as part of the state and do more interesting thigs with it. 

Finally we will implement a score_document method, that maps a document to a non-negative real number. This significance of this will become clear shortly. 

In [12]:
class LTSUserState(user.AbstractUserState):
    
    def __init__(self, current, time_budget=1):

        np.random.seed(1)
        W = generate_W()
        ## State vavriables
        ##############################
        self.current = current
        self.time_budget = time_budget

    
    def create_observation(self):
        return np.array([self.current,])

    @staticmethod
    def observation_space():
        return spaces.Discrete(LTSUserState.doc_num)

    
    # scoring function for use in the choice model -- the user is more likely to
    # click on more chocolatey content.
    def score_document(self, doc_obs):
#         print(self.current, doc_obs[0], LTSUserState.doc_num)
        return W[self.current, doc_obs[0]]

Also similarly to our document model, we have need a starting state sampler, that sets the starting user state for every session. For this tutoral, we will just sample the starting $\text{nke}_0$ and keep all the static parameters the same, meaning that we essentially deal with the same user at different levels of satisfaction. One can, of course, extend this easily to generate also users with different parameters by randomizing the values. 

Observe that if $\eta = 0$, $\text{nke}$ would be bounded in the interval $\left[-\frac{1}{1-\beta}, \ldots, \frac{1}{1-\beta} \right]$ at all times, so as starting distribution we just sample uniformly from that range. Sampling code has to be implemented in sample_user(), as required by the base class.

In [13]:
class LTSStaticUserSampler(user.AbstractUserSampler):
    _state_parameters = None

    def __init__(self, user_ctor=LTSUserState, doc_num=-10, current=0, **kwargs):
        user_ctor.doc_num = doc_num
        self.doc_num = doc_num
        self._state_parameters = {'current': current}
        super(LTSStaticUserSampler, self).__init__(user_ctor, **kwargs)

    def sample_user(self):
        current = np.random.randint(self.doc_num)
        self._state_parameters['current'] = current
        return self._user_ctor(**self._state_parameters)


Let's try this out!

In [14]:
sampler = LTSStaticUserSampler(doc_num=10)
starting_nke = []
for i in range(1000):
    sampled_user = sampler.sample_user()

## Response model

The next thing we want to check off our list is the user response class. RecSim will generate one response for every recommended item in the slate. The contents of the response are what the agent will see as document-specific feedback from the recommendation (the non-document specific feedback being generated in LTSUserState.create_observation).  

In [49]:
class LTSResponse(user.AbstractResponse):

    def __init__(self, clicked=False):
        self.clicked = clicked

    def create_observation(self):
        print(self.clicked)
        return {'click': int(self.clicked)}

    @classmethod
    def response_space(cls):
        return spaces.Dict({
            'click':
                spaces.Discrete(2),
        })

##User model

Now that we have a way to generate users for our sessions, need to specify the actual user behavior. A RecSim user model (deriving from recsim.user.AbstractUserModel) is responsible for 
* maintaining user state, 
* evolving user state as a result of recommendations,
* generating a response to a slate of recommendations.

To this end, our user model is required by the base class to implement update_state() and simulate_response(), as well as is_terminal, which indicates when the end of the session occurs. This is facilitated by decreasing *self.time_budget* on every step.To make presentation clearer, we will define each function separately, and then assemble them into a class.



Our init is simple---we just pass the response_model constructor, user sampler and slate size down to the AbstractUserModel base class. Exploring other environments, the reader might notice that user model __init__ functions do offer a lot of flexibility for configuring the simulation. For now, however, we stick to the basics and hardcode things.

In [50]:
def user_init(self,
              slate_size,
              doc_num, choice_features,
              seed=0):

    LTSUserStateClass = LTSUserState
    LTSUserStateClass.doc_num = doc_num
    
    super(LTSUserModel,
        self).__init__(LTSResponse,
                       LTSStaticUserSampler(LTSUserState,
                                            seed=seed, doc_num=DOC_NUM), slate_size)
    self.choice_model = UserChoiceModel(choice_features)

The simulate_response() method takes in a slate (list) of recommended (i.e., produced by the agent) LTSDocuments and must output a slate of user responses. The *k*-th response in the slate of responses corresponds to the *k*-th document in the recommendation slate. In this case, we pick one document to click on based on our choice model, and produce an engagement value. We will let the responses to the unclicked documents be vacuous, however, one might use them in more subtle ways (e.g., recording whether the user inspected that document, etc.).  

In [51]:
from recsim.choice_model import NormalizableChoiceModel, softmax

In [52]:
class UserChoiceModel(NormalizableChoiceModel):  # pytype: disable=ignored-metaclass
    """Abstract class to represent the user choice model.
    Each user has a choice model.
    """
    def __init__(self, choice_features):
        super(UserChoiceModel, self).__init__()
        self._no_click_mass = choice_features.get('no_click_mass', -float('Inf'))

    def score_documents(self, user_state, doc_obs):
        logits = self._score_documents_helper(user_state, doc_obs)
        logits = np.append(logits, self._no_click_mass)

        all_scores = softmax(logits)
        self._scores = all_scores[:-1]
        self._score_no_click = all_scores[-1]

In [53]:
def simulate_response(self, slate_documents):
    # List of empty responses
    responses = [self._response_model_ctor() for _ in slate_documents]
    # Get click from of choice model.
    self.choice_model.score_documents(self._user_state, [doc.create_observation() for doc in slate_documents])
    scores = self.choice_model.scores
    selected_index = self.choice_model.choose_item()
    # Populate clicked item.
#    selected_index.remove(None)
    if not selected_index:
        return responses
    self._generate_response(slate_documents[selected_index],
                          responses[selected_index])
    return responses

def generate_response(self, doc, response):
    '''doc'''
    response.clicked = True


The update_state() method implements our state transition kernel. It consumes the recommended slate, as well as the actual choices (responses) to induce a state transition. The state is modified in-place, so the function does not return anything.

In [54]:
def update_state(self, slate_documents, responses):
    for doc, response in zip(slate_documents, responses):
        if response.clicked:
            self._user_state.time_budget -= np.random.binomial(1, 0.1)
            return
        else:
            self._user_state.time_budget -= np.random.binomial(1, 0.2)

Finally, the session expires when the time budget goes to 0.

In [55]:
def is_terminal(self):
  """Returns a boolean indicating if the session is over."""
  return self._user_state.time_budget <= 0


We have all the components to instantiate a user model, so let's wrap them up in a class.

In [56]:
LTSUserModel = type("LTSUserModel", (user.AbstractUserModel,),
                    {"__init__": user_init,
                     "is_terminal": is_terminal,
                     "update_state": update_state,
                     "simulate_response": simulate_response,
                     "_generate_response": generate_response})

Finally, we assemble all components into an Environment.

In [102]:
 slate_size = 3
 num_candidates = 4
 ltsenv = environment.Environment(
            LTSUserModel(slate_size, choice_features={'no_click_mass': -1}, doc_num=DOC_NUM),
            LTSDocumentSampler(doc_num=DOC_NUM),
            num_candidates,
            slate_size,
            resample_documents=True)


In [103]:
def clicked_engagement_reward(responses):
  reward = 0.0
  for response in responses:
    if response.clicked:
      reward += 1
#  print(responses)
  return reward

Now, we simply use the OpenAI gym wrapper, which essentially provides a familiar step-based API.

In [104]:
lts_gym_env = recsim_gym.RecSimGymEnv(ltsenv, clicked_engagement_reward)

In [105]:
np.random.seed(100004)

In [106]:
observation_0 = lts_gym_env.reset()
print('Observation 0')
print('Available documents')
doc_strings = ['doc_id ' + key + " kaleness " + str(value) for key, value
               in observation_0['doc'].items()]
print('\n'.join(doc_strings))
print('Noisy user state observation')
print(observation_0['user'])

for i in range(10):
    # Agent recommends the first three documents.
    recommendation_slate_0 = [0, 1, 2]
    observation_1, reward, done, _ = lts_gym_env.step(recommendation_slate_0)
    print('Observation 1')
    print('Available documents')
    doc_strings = ['doc_id ' + key + " kaleness " + str(value) for key, value
                   in observation_1['doc'].items()]
    print('\n'.join(doc_strings))
    rsp_strings = [str(response) for response in observation_1['response']]
    print('User responses to documents in the slate')
    print('\n'.join(rsp_strings))
    print('Noisy user state observation')
    print(observation_1['user'])

Observation 0
Available documents
doc_id 4 kaleness [4]
doc_id 0 kaleness [0]
doc_id 1 kaleness [1]
doc_id 2 kaleness [2]
Noisy user state observation
[1]
Debug in RecSim source ----------------------
Probs for current slate:
[0.35959013 0.27721419 0.26551732 0.09767836]
---------------------------------------------
False
False
True
Observation 1
Available documents
doc_id 3 kaleness [3]
doc_id 4 kaleness [4]
doc_id 0 kaleness [0]
doc_id 1 kaleness [1]
User responses to documents in the slate
{'click': 0}
{'click': 0}
{'click': 1}
Noisy user state observation
[1]
Debug in RecSim source ----------------------
Probs for current slate:
[0.31350729 0.33609506 0.25910144 0.09129621]
---------------------------------------------
False
False
True
Observation 1
Available documents
doc_id 2 kaleness [2]
doc_id 3 kaleness [3]
doc_id 4 kaleness [4]
doc_id 0 kaleness [0]
User responses to documents in the slate
{'click': 0}
{'click': 0}
{'click': 1}
Noisy user state observation
[1]
Debug in RecSim

In [107]:
observation_0['doc'].items()

odict_items([('4', array([4])), ('0', array([0])), ('1', array([1])), ('2', array([2]))])

In [108]:
from recsim.agents.full_slate_q_agent import FullSlateQAgent
from recsim.simulator import runner_lib

def create_agent(sess, environment, eval_mode, summary_writer=None):
    return FullSlateQAgent(sess, environment.observation_space, environment.action_space,) 


In [109]:
slate_size = 3
num_candidates = 5
ltsenv = environment.Environment(
        LTSUserModel(slate_size, choice_features={'no_click_mass': 1}, doc_num=DOC_NUM),
        LTSDocumentSampler(doc_num=DOC_NUM),
        num_candidates,
        slate_size,
        resample_documents=False)

env = recsim_gym.RecSimGymEnv(ltsenv, clicked_engagement_reward)

tmp_base_dir = 'tmp'
episode_log_file_train = 'episodes_train'

In [110]:
import numpy as np
from recsim.agents import full_slate_q_agent
from recsim.simulator import runner_lib

seed = 0
slate_size = 1
np.random.seed(seed)
env_config = {
  'num_candidates': 5,
  'slate_size': slate_size,
  'resample_documents': True,
  'seed': seed,
}

In [111]:
! rm -rf tmp

runner = runner_lib.TrainRunner(
  base_dir=tmp_base_dir,
  create_agent_fn=create_agent,
#  env=interest_evolution.create_environment(env_config),
  env=env,
  episode_log_file=episode_log_file_train,
  max_training_steps=50,
  num_iterations=10)
runner.run_experiment()

runner = runner_lib.EvalRunner(
  base_dir=tmp_base_dir,
  create_agent_fn=create_agent,
  env=env,
  max_eval_episodes=5,
  test_mode=True)

runner.run_experiment()

INFO:tensorflow:max_training_steps = 50, number_iterations = 10,checkpoint frequency = 1 iterations.
INFO:tensorflow:max_steps_per_episode = 27000
INFO:tensorflow:Creating FullSlateQAgent agent with the following parameters:
INFO:tensorflow:	 gamma: 0.990000
INFO:tensorflow:	 update_horizon: 1.000000
INFO:tensorflow:	 min_replay_history: 20000
INFO:tensorflow:	 update_period: 4
INFO:tensorflow:	 target_update_period: 8000
INFO:tensorflow:	 epsilon_train: 0.010000
INFO:tensorflow:	 epsilon_eval: 0.001000
INFO:tensorflow:	 epsilon_decay_period: 250000
INFO:tensorflow:	 tf_device: /cpu:*
INFO:tensorflow:	 use_staging: True
INFO:tensorflow:	 optimizer: <tensorflow.python.training.rmsprop.RMSPropOptimizer object at 0x15a0ffdd0>
INFO:tensorflow:	 max_tf_checkpoints_to_keep: 4
INFO:tensorflow:Creating a OutOfGraphReplayBuffer replay memory with the following parameters:
INFO:tensorflow:	 observation_shape: (6, 5)
INFO:tensorflow:	 observation_dtype: float32
INFO:tensorflow:	 terminal_dtype: <

Debug in RecSim source ----------------------
Probs for current slate:
[0.15727407 0.20400905 0.22924036 0.40947652]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.17326908 0.20965193 0.1659581  0.45112089]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.15913098 0.1661412  0.24216496 0.43256286]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.19938595 0.21375144 0.15783167 0.42903095]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.2214126  0.16348858 0.17069077 0.44440805]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.15376666 0.23400154 0

Debug in RecSim source ----------------------
Probs for current slate:
[0.19938595 0.21375144 0.15783167 0.42903095]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.15946039 0.2324271  0.19294371 0.41516879]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.23400154 0.15376666 0.1942507  0.4179811 ]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.20537191 0.23077178 0.15164432 0.41221199]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.23077178 0.20537191 0.15164432 0.41221199]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.17326908 0.1659581  0

Debug in RecSim source ----------------------
Probs for current slate:
[0.20400905 0.15727407 0.22924036 0.40947652]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.24216496 0.15913098 0.1661412  0.43256286]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.22924036 0.20400905 0.15727407 0.40947652]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.2324271  0.19294371 0.15946039 0.41516879]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.2214126  0.17069077 0.16348858 0.44440805]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.21227549 0.16364681 0

Debug in RecSim source ----------------------
Probs for current slate:
[0.1659581  0.17326908 0.20965193 0.45112089]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.22191189 0.18421476 0.19748718 0.39638617]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.16364681 0.21227549 0.1980092  0.4260685 ]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.23400154 0.15376666 0.1942507  0.4179811 ]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.20400905 0.15727407 0.22924036 0.40947652]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.23400154 0.15376666 0

...

Debug in RecSim source ----------------------
Probs for current slate:
[0.2214126  0.16348858 0.17069077 0.44440805]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.22924036 0.15727407 0.20400905 0.40947652]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.16348858 0.2214126  0.17069077 0.44440805]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.20537191 0.23077178 0.15164432 0.41221199]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.23400154 0.15376666 0.1942507  0.4179811 ]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.2214126  0.16348858 0

Debug in RecSim source ----------------------
Probs for current slate:
[0.15727407 0.20400905 0.22924036 0.40947652]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.20965193 0.1659581  0.17326908 0.45112089]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.15376666 0.23400154 0.1942507  0.4179811 ]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.20400905 0.15727407 0.22924036 0.40947652]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.1980092  0.21227549 0.16364681 0.4260685 ]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.22924036 0.20400905 0

Debug in RecSim source ----------------------
Probs for current slate:
[0.2214126  0.16348858 0.17069077 0.44440805]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.1980092  0.21227549 0.16364681 0.4260685 ]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.15946039 0.2324271  0.19294371 0.41516879]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.2214126  0.16348858 0.17069077 0.44440805]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.15913098 0.1661412  0.24216496 0.43256286]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.19938595 0.21375144 0

Debug in RecSim source ----------------------
Probs for current slate:
[0.16348858 0.2214126  0.17069077 0.44440805]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.19938595 0.21375144 0.15783167 0.42903095]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.1980092  0.21227549 0.16364681 0.4260685 ]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.1659581  0.20965193 0.17326908 0.45112089]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.15376666 0.1942507  0.23400154 0.4179811 ]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.1980092  0.21227549 0

Debug in RecSim source ----------------------
Probs for current slate:
[0.16364681 0.21227549 0.1980092  0.4260685 ]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.19748718 0.22191189 0.18421476 0.39638617]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.19294371 0.2324271  0.15946039 0.41516879]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.2214126  0.17069077 0.16348858 0.44440805]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.21227549 0.16364681 0.1980092  0.4260685 ]
---------------------------------------------
False
False
False
Debug in RecSim source ----------------------
Probs for current slate:
[0.20400905 0.22924036 0

In [35]:
!kill 65553
%load_ext tensorboard

/bin/sh: line 0: kill: (65553) - No such process


In [ ]:
%tensorboard --logdir tmp